In [1]:
import pandas as pd
import numpy as np

# make sure the aps module is in the pythonpath
import sys
from pathlib import Path

APS_dir = str(Path.cwd().parents[1])
if APS_dir not in sys.path:
    sys.path.append(APS_dir)
import aps.aps_io.get_forecasts as gf

# Get API data
Request necessary data from https://api01.nve.no/hydrology/forecast/avalanche/v4.0.0/api

Explanation at http://api.nve.no/doc/snoeskredvarsel/#format

Insert relevant data into SQLite database.

General:
- date
- region
- region-ID
- danger level

Avalanche problem:
- avalanche problem name
- avalanche problem order
- trigger
- probability
- elevation

Mountain weather:
- avg. precipitation
- max precipitation (most exposed area)
- temperature (at diff. levels)
- wind speed
- wind direction

In [2]:
warnings, url = gf.get_warnings_as_json(3024, "2018-01-01", "2018-02-01", lang_key=1, simple=False, recursive_count=5)
print(url)

df = pd.DataFrame(warnings)
df.head(5)

https://api01.nve.no/hydrology/forecast/avalanche/v4.0.0/api/AvalancheWarningByRegion/Detail/3024/1/2018-01-01/2018-02-01


,Author,AvalancheDanger,AvalancheProblems,CountyList,CurrentWeaklayers,DangerLevel,DangerLevelName,EmergencyWarning,LangKey,LatestAvalancheActivity,...,RegionId,RegionName,RegionTypeId,RegionTypeName,SnowSurface,UtmEast,UtmNorth,UtmZone,ValidFrom,ValidTo
0,jostein@nve,Små nebørsmengder vil ikkje føre til noko vese...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",28.12 vert det observert overflaterim. Rimet f...,2,2 Moderat,Ikke gitt,1,None,...,3024,Sunnmøre,10,A,I høgfjellet er snøoverflata stort sett hard o...,62473,6916553,33,2018-01-01T00:00:00,2018-01-01T23:59:59
1,jostein@nve,Små nebørsmengder vil ikkje føre til noko vese...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",28.12 vert det observert overflaterim. Rimet f...,2,2 Moderat,Ikke gitt,1,30.12 gjekk det eit skred ved Lafjellet.,...,3024,Sunnmøre,10,A,I høgfjellet er snøoverflata stort sett hard o...,62473,6916553,33,2018-01-02T00:00:00,2018-01-02T23:59:59
2,Halvor@NVE,Perioder med litt snø og vind de siste dagene ...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",Det er observert overflaterim i regionen.,2,2 Moderat,Ikke gitt,1,Det er ikke registrert skredaktivitet de siste...,...,3024,Sunnmøre,10,A,Det har kommet 45-70 cm snø i dagene etter mil...,62473,6916553,33,2018-01-03T00:00:00,2018-01-03T23:59:59
3,Halvor@NVE,Perioder med litt snø og vind de siste dagene ...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",Det er observert overflaterim flere steder i r...,2,2 Moderat,Ikke gitt,1,Det er ikke registrert skredaktivitet de siste...,...,3024,Sunnmøre,10,A,Det kom 45-70 cm snø i dagene etter mildværet ...,62473,6916553,33,2018-01-04T00:00:00,2018-01-04T23:59:59
4,Halvor@NVE,Perioder med litt snø og vind de siste dagene ...,"[{'AvalancheProblemId': 1, 'AvalancheExtId': 2...","[{'Id': '15', 'Name': 'Møre og Romsdal'}, {'Id...",Det er observert overflaterim flere steder i r...,2,2 Moderat,Ikke gitt,1,Det er ikke registrert skredaktivitet de siste...,...,3024,Sunnmøre,10,A,23.12 var det mildvær og regn til topps. Deret...,62473,6916553,33,2018-01-05T00:00:00,2018-01-05T23:59:59


In [3]:
df.columns
df['AvalancheProblems'][23]

[{'AvalCauseId': 10,
  'AvalCauseName': 'Nedføyket svakt lag med nysnø',
  'AvalProbabilityId': 3,
  'AvalProbabilityName': 'Mulig ',
  'AvalPropagationId': 2,
  'AvalPropagationName': 'Noen bratte heng',
  'AvalTriggerSimpleId': 21,
  'AvalTriggerSimpleName': 'Liten tilleggsbelastning',
  'AvalancheAdvice': 'Vær forsiktig i  bratte heng under og etter snøfallet til nysnøen har stabilisert seg. Hold avstand til hverandre ved ferdsel i bratt terreng. Unngå ferdsel i / ved terrengfeller. Skredproblemet finnes overalt hvor det ligger mye nysnø i bratt terreng. Se etter nysnø som binder seg sammen til myke flak og sprekker opp eller fester seg dårlig til den eldre snøen under. Snø som sprekker opp rundt skiene/brettet er et typisk tegn.',
  'AvalancheExtId': 20,
  'AvalancheExtName': 'Tørre flakskred',
  'AvalancheProblemId': 1,
  'AvalancheProblemTypeId': 7,
  'AvalancheProblemTypeName': 'Nysnøflak',
  'AvalancheTypeId': 10,
  'AvalancheTypeName': 'Flakskred',
  'DestructiveSizeExtId': 2,

In [20]:
df['MountainWeather'][23]

{'CloudCoverId': 30,
 'CloudCoverName': 'Skyet',
 'Comment': 'Økende nedbør fra sørvest om ettermiddagen.',
 'LastSavedTime': '2018-01-23T15:49:58.697',
 'MeasurementTypes': [{'Id': 50,
   'MeasurementSubTypes': [{'Id': 110,
     'Name': 'Tidsperiode slutt',
     'SortOrder': 110,
     'Value': '18'},
    {'Id': 100, 'Name': 'Tidsperiode start', 'SortOrder': 100, 'Value': '12'},
    {'Id': 90, 'Name': 'moh', 'SortOrder': 90, 'Value': '800'}],
   'Name': 'Nullgradersgrense',
   'SortOrder': 50},
  {'Id': 40,
   'MeasurementSubTypes': [{'Id': 90,
     'Name': 'moh',
     'SortOrder': 90,
     'Value': '1400'},
    {'Id': 40, 'Name': 'Maks', 'SortOrder': 40, 'Value': None},
    {'Id': 30, 'Name': 'Min', 'SortOrder': 30, 'Value': None}],
   'Name': 'Temperatur',
   'SortOrder': 40},
  {'Id': 30,
   'MeasurementSubTypes': [{'Id': 110,
     'Name': 'Tidsperiode slutt',
     'SortOrder': 110,
     'Value': None},
    {'Id': 100, 'Name': 'Tidsperiode start', 'SortOrder': 100, 'Value': None},
 

In [4]:
df_a = df

In [5]:
df_a['AP1.CauseId'] = np.nan
df_a['AP2.CauseId'] = np.nan
df_a['AP3.CauseId'] = np.nan

df_a['AP1.TypeId'] = np.nan
df_a['AP2.TypeId'] = np.nan
df_a['AP3.TypeId'] = np.nan

df_a.columns

Index(['Author', 'AvalancheDanger', 'AvalancheProblems', 'CountyList',
       'CurrentWeaklayers', 'DangerLevel', 'DangerLevelName',
       'EmergencyWarning', 'LangKey', 'LatestAvalancheActivity',
       'LatestObservations', 'MainText', 'MountainWeather', 'MunicipalityList',
       'NextWarningTime', 'PreviousWarningRegId', 'PublishTime', 'RegId',
       'RegionId', 'RegionName', 'RegionTypeId', 'RegionTypeName',
       'SnowSurface', 'UtmEast', 'UtmNorth', 'UtmZone', 'ValidFrom', 'ValidTo',
       'AP1.CauseId', 'AP2.CauseId', 'AP3.CauseId', 'AP1.TypeId', 'AP2.TypeId',
       'AP3.TypeId'],
      dtype='object')

In [6]:
for index, row in df_a.iterrows():
    for i in range(len(row['AvalancheProblems'])):
        column_name_prefix = "AP{0}".format(row['AvalancheProblems'][i]['AvalancheProblemId'])
        df_a.loc[index, '{0}.CauseId'.format(column_name_prefix)] = row['AvalancheProblems'][i]['AvalCauseId']
        df_a.loc[index, '{0}.TypeId'.format(column_name_prefix)] = row['AvalancheProblems'][i]['AvalancheProblemTypeId']

### List relevant parameters for the regression analysis

See full list at http://api.nve.no/doc/snoeskredvarsel/#format

- Precipitation (most exposed): MeasurementType=10; MeasurementSubType=60
- Wind speed: MeasurementType=20; MeasurementSubType=20
- Wind direction: MeasurementType=20; MeasurementSubType=50

In [ ]:
for dt,DL, APs in zip(df['ValidFrom'], df['DangerLevel'],df['AvalancheProblems']):
    for AP in APs:
        print(dt, DL, AP['AvalancheProblemTypeName'], AP['DestructiveSizeExtName'], AP['AvalTriggerSimpleName'], AP['AvalancheProblemId'])

In [19]:
df_b = df_a[df_a["AP1.TypeId"] == 7]
df_b[['ValidFrom', 'DangerLevel', 'AP1.CauseId']]

,ValidFrom,DangerLevel,AP1.CauseId
22,2018-01-23T00:00:00,2,10.0
23,2018-01-24T00:00:00,2,10.0
24,2018-01-25T00:00:00,3,10.0
25,2018-01-26T00:00:00,2,10.0
27,2018-01-28T00:00:00,3,10.0
29,2018-01-30T00:00:00,2,10.0
